In [2]:
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
from api import static

In [5]:
live = static()
elements_df = live.get_elements_summary()

In [6]:
elements_df['now_cost'] = elements_df['now_cost'].astype('int64')

In [7]:
cost_columns = ['code', 'web_name','cost_change_event', 'cost_change_event_fall', 'cost_change_start',
                'cost_change_start_fall', 'now_cost']

In [8]:
cost_changes_now = elements_df.loc[elements_df.cost_change_event != 0, cost_columns]

In [19]:
to_export = cost_changes_now.copy()
to_export['now_cost'] = to_export['now_cost'].apply(lambda x: x*0.1)
to_export.to_excel(rf'D:\project\footystats\changes\changes_{now.date()}.xlsx',index=None)

In [13]:
import datetime
now = datetime.datetime.now()
print(now)

2023-11-12 15:03:07.132048


In [21]:
print(now)

15:03:07.132048
